<h3><center><u>Kaggle Data Challenge</u></center></h3>
<center><b>Wyatt Priddy</b></center>

<b>Problem:</b>
<br><br>
<i>You are consulting for a real estate company that has a niche in purchasing properties to
rent out short-term as part of their business model specifically within New York City. The
real estate company has already concluded that two bedroom properties are the most
profitable; however, they do not know which zip codes are the best to invest in.
The real estate company has engaged your firm to build out a data product and provide
your conclusions to help them understand which zip codes would generate the most
profit on short term rentals within New York City.</i>
<br><br>
<b>Solution:</b>

<text> Zipcode<b> 10312</b> in Staten Island provides the greatest rate of return on investment as it becomes profitable in just <i>under 6 years</i> based on a 75% occupancy rate. This zipcode overwhelmingly outperforms all of the provided zipcodes where, of the data provided for May 2017, the <b>average time to become profitable is approximately 20 years</b> from investment. The second most profitable zipcode is 10304 (also located in Staten Island), which becomes profitable in <i>approximately 12 years</i>.</text>
<br><br>

In [16]:
#  Imports
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets  # Supplies interactive component
import matplotlib.pyplot as plt

<h3>Data Extraction and Transformation:</h3>
<br><br>
<b>AirBnB Data Quality:</b>
<br><br>
<text>&emsp;1. The data for zipcodes was inconsistent in its length and values. The assumption was made that the analysis is based off standard zipcodes. Therefore, only the first 5 digits were kept.</text>
<br><br>
<text>&emsp;2. The daily price of a short rental AirBnB was stored in text format. In order to effectively analyze the data it was converted to a float using regex.</text>
<br><br>
<text>&emsp;3. The common unit of time was established using the Zillow dataset as the template at the month level. A more datetime-specific analysis would require more detailed data and additional frameworks to be implemented. 

In [17]:
#  Read in the AirBnB Data Set

airbnb_data = pd.read_csv('airbnb_data.csv', low_memory=False)
        
#  If more granularity needed than a month, then additional data and frameworks would be implemented

airbnb_data.last_scraped = airbnb_data.last_scraped.apply(lambda date:
                                "20" + date.split('/')[-1] + '-' + date.split('/')[0].rjust(2, chr(48)))
                                
#  Clean zipcodes
airbnb_data.zipcode = airbnb_data.zipcode.apply(lambda zipcode: str(zipcode)[:5] if len(str(zipcode)[:5]) == 5 else np.nan)

#  Convert price
airbnb_data.price = airbnb_data.price.apply(lambda price: float(re.sub("[^\d\.]", "", price)))

#  Subsect DataFrame
airbnb_data = airbnb_data[['last_scraped', 'city', 'state', 'bedrooms', 'zipcode', 'price', 'neighbourhood_group_cleansed']]

#  Rename columns
airbnb_data.columns = ['Date', 'City', 'State', 'bedrooms', 'zipcode', 'price', 'Borough']

<b>Zillow Data Quality:</b>
<br><br>
<text>&emsp;1. While cleaner than the AirBnB dataset, the Zillow data set has a fundamental data quality issue. For quality analysis, we want rows rather than columns. To achieve a more ideal data set, the datetime data is transformed into a single column - "Date".</text>
<br><br>
<text>&emsp;2. The zipcodes are far cleaner within the Zillow data set. However, the data is stored as an integer therefore, it needs to be made categorical for accurate analysis.</text>
<br><br>
<text>&emsp;3. MedianPrice within the Zillow data set contains 266141 NaN values. As this is the main comparative metric, those data points that do not contain this information need to be removed. </text>

In [18]:
zillow_data = pd.read_csv('zillow_data.csv')

#  Melt date columns
zillow_data_clean = pd.melt(zillow_data, id_vars=zillow_data.columns[:7], var_name='Date', value_name = 'MedianPrice')

#  Drop nulls
zillow_data_clean = zillow_data_clean[zillow_data_clean['MedianPrice'].notna()].reset_index(drop = True)

#  Make zipcode categorical
zillow_data_clean.RegionName = zillow_data_clean.RegionName.astype(str)

#  Subsect DataFrame
zillow_data_clean = zillow_data_clean[['RegionName', 'Date', 'MedianPrice', 'City', 'State']]

#  Rename columns
zillow_data_clean.columns = ['zipcode']+[*zillow_data_clean.columns[1:]]

<h3>Analysis Parameters:</h3>
<br><br>
<text>Data pulled in the analysis below is based on the contents of <b>search_terms</b>. The analysis takes four parameters: <br><br>
    &emsp; 1. <b>city</b>: name of city to query*
<br>
    &emsp; 2. <b>state</b>: abbreviated state name of corresponding city*
<br>
    &emsp; 3. <b>bed_rooms</b>: number of bedrooms in the property    
<br>
    &emsp; 4. <b>occupancy_rate</b>: percentage of time residence is occupied (0 - 100)




&emsp;<i>*Currently does not have dynamic functionality due to limited data</i>
</text>

In [19]:
#  UPDATE THESE FOR DIFFERENT METRIC OUTCOMES

search_terms = {'city': 'New York',
               'state': 'NY',
               'bed_rooms': 2,
               'occupancy_rate': 75}

<h3>Merging Data and Additional Metrics for Analysis</h3>
<br><br>
<text><b>pull_data</b> takes the parameters of the <b>search_terms</b> and transforms the data for analysis in five fundamental steps:
<br><br>
    &emsp;1. Queries the Zillow dataset to include just the relevant <b>city</b> and <b>state</b>
<br><br>
    &emsp;2. Takes the unique <b>zipcodes</b> of the queried Zillow dataset and the parameter <b>bed_rooms</b> and filters the AirBnB dataset so that only comparable data points are included in the analysis 
<br><br>
&emsp;3. Merges the Zillow dataset into the AirBnB dataset for ease of analysis
<br><br>
&emsp;4. Creates custom metric <b>revenue_per_year</b> to help calculate zipcode profitability
<br><br> 
$$ Revenue Per Year = Daily Price * 365 * \Bigg (\frac {Occupancy Rate} {100} \Bigg )$$
<br>
&emsp;5. Averages data based on zipcode to create clean dataset for analysis

In [20]:
def pull_data(city: str, state: str, bed_rooms: int, occupancy_rate: int=75) -> pd.DataFrame:
    """
    Reads and transforms cleaned data set for later analysis.
    
    Parameters
    ----------
    city: city name
    state: state abbreviation
    bed_rooms: number of bedrooms available
    occupancy_rate: yearly rate of occupancy in (0 - 100)
    
    
    Special Consideration
    ---------
    global location and bedrooms for access outside function
    
    """
    
    global location, bedrooms
    
    location = city + ', ' + state
    
    bedrooms = bed_rooms
    
    #  Query Zillow Data 
    zillow_filtered = zillow_data_clean[(zillow_data_clean.City == city) &
                                        (zillow_data_clean.State == state)]
    
    #  Keeping only intersecting Dates
    zillow_filtered = zillow_filtered[zillow_filtered.Date.isin(airbnb_data.Date.unique())]

    unique_zipcodes = [*map(lambda code: str(code), zillow_filtered.zipcode.unique())]
    
    #  Query AirBnB Data
    airbnb_filtered = airbnb_data[(airbnb_data.zipcode.isin(unique_zipcodes)) & (airbnb_data.bedrooms == float(bed_rooms))]
    
    merged_df = airbnb_filtered.merge(zillow_filtered, how = 'left', left_on = 'zipcode', right_on = 'zipcode').reset_index(drop = True)

    merged_df = merged_df.drop(['Date_y', 'City_y', 'State_y'], axis = 1)
    
    #  Custom Metric to Measure Yearly Revenue
    merged_df['revenue_per_year'] = round(merged_df.price * (365 * (occupancy_rate/100)), 2)
    
    merged_df.columns = ['Date', 'City', 'State', 'bedrooms', 'zipcode', 'price', 'Borough', 'MedianPrice', 'revenue_per_year']
    
    #  Final Group for Analysis
    final_stack = merged_df.groupby(['Borough', 'Date','zipcode']).mean().astype(float).reset_index()
    
    return final_stack

<h3>Visual Narrative (1/3)</h3>
<br>
<text><h4>3 key visual narratives were dynamically applied to the transformed data set:</h4>
<br><br>
    &emsp;1. <b>Double Lollipop Graph</b>
    <br><br>&emsp;&emsp;This interactive graph allows for a dynamic analysis based on the parameters in the <b>search_terms</b> and creates a visual analysis of the difference between the <b>MedianPrice</b> of a property and <b>accumulated_revenue</b> over a user-defined period of years.
    <br><br>
    &emsp;&emsp;The two interactive features are:
    <br><br>
    &emsp;&emsp;&emsp;1. years: allows user-defined period of time to analyze projected revenues and easily see when a zipcode becomes profitable
    <br><br>
    &emsp;&emsp;&emsp;2. borough: allows toggling between different boroughs within NYC*
    <br><br>
    &emsp;&emsp;A custom metric was created to see revenue projections based on years since investment:
   <br><br>
    $$ Accumulated Revenue = Revenue Per Year * Years $$   
    <br>
</text>
<i>*borough breakout will need to be adjusted in the future to account for more AirBnB data within different cities</i>

In [21]:
def lollipop(data:pd.DataFrame, value_1: str, value_2:str, years: float, borough = 'all'):
    """
    Creates Lollipop Graph for Dynamic Visual Analysis
    
    Parameters
    ----------
    data: Returned from pull_data function
    value_1: Name of column for data point 1
    value_2: Name of column for data point 2
    years: metric used for forecasting revenue
    borough: subsections of selected city

    """

    #  Dynamic Condition to subsect data
    if borough.lower() != 'all':
        
        data = data[data.Borough == borough]
    
    if years < 0:
        
        return 'Please only use positive numbers'
        
    df_sorted = data.sort_values(by=value_1)

    chart_range = range(1, df_sorted.shape[0]+1)
    
    plt.figure(figsize=(10,8))
    
    #  Creates horizontal lollipop stick
    plt.hlines(y=chart_range, xmin=df_sorted[value_2], xmax=df_sorted[value_1]*years, color='grey', alpha=0.4)
    
    #  Plot Corresponding Data Points
    plt.scatter(df_sorted[value_2], chart_range, color='blue', alpha=1, label=value_2)

    plt.scatter(df_sorted[value_1]*years, chart_range, color='red', alpha=0.6, label='accumulated_revenue')
    
    plt.legend()
 
    plt.yticks(chart_range, df_sorted['zipcode'])

    #  Dynamic Title 
    plt.title(f"Revenue vs Cost over {years} years for {bedrooms} bedrooms in {location}", loc='center')
    
    plt.xlabel(f'Dollars ($USD)')
    
    #  Scalable xticks rounded to the 10,000's place
    xtick_values = [*range(0, n := int(max(df_sorted[value_2])), round(n //8, -5))]

    xtick_labels = ["${:,}".format(i) for i in xtick_values]
    
    plt.xticks(xtick_values, xtick_labels, rotation = 45)
    
    plt.ylabel(f'{location} Zipcodes')

    plt.show()
    
    
#  Creates Interact features on Graph
_ = widgets.interact(
    lollipop, 
    data = widgets.fixed(pull_data(**search_terms)),
    value_1 = widgets.fixed('revenue_per_year'), 
    value_2 = widgets.fixed('MedianPrice'), 
    years = 10,
    borough = ['All', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']
)

interactive(children=(IntSlider(value=10, description='years', max=30, min=-10), Dropdown(description='borough…

<h3>Visual Narrative (2/3)</h3>
<br><br>
<text>
    2. <b>Dynamic Bar Graph</b> 
    <br><br>
    This interactive graph allows for a dynamic analysis based on the parameters in the <b>search_terms</b> and allows tracking <b>accumulated revenue</b> as compared to <b>median price</b> with an additional metric calculating the <b>profitability</b> over time. 
<br><br>
&emsp;&emsp;There are two interactive features:
<br><br>
&emsp;&emsp;&emsp;1. years: allows user-defined period of time to analyze projected revenues and easily see when a zipcode becomes profitable
<br><br>
&emsp;&emsp;&emsp;2. level of profit: while not inherently user-defined, this graph automatically sorts the visualization by most profitable zipcode based on the year value.
<br><br>
&emsp;&emsp;Custom metrics were created to see revenue and profit projections based on years since investment:
<br><br>
    $$ Accumulated Revenue = Revenue Per Year * Years $$
<br>
    <center>------------</center>
<br>
    $$ Profit = Accumulated Revenue - Median Price$$
</text>

In [22]:
@widgets.interact(data = widgets.fixed(pull_data(**search_terms)),
                  years = 10)
def breakeven_visual(data, years):
    """
    Creates Dynamic Visualization of Break Even Analysis
    
    Parameters
    ----------
    
    data: Returned from pull_data function
    years: Dynamic Metric for calculating projected profitability
    """
    
    if years < 0:
        
        return 'Please only use positive numbers'
    
    data['Accumulated Revenue'] = round(data.revenue_per_year * years, 2)
    
    data['Profit'] = [round(profit,2) if (profit := data['Accumulated Revenue'][i] - data['MedianPrice'][i]) > 0 else
                     0 for i in range(data.shape[0])]
    
    #  Dynamic Plot Axis by Profit
    data = data.sort_values(by='Profit', ascending= False)

    ind = np.arange(len(data['Accumulated Revenue'])) 
    
    width = 0.3  

    fig, ax = plt.subplots(figsize = (12, 8))

    #  Plots Profitability Metrics by Zipcode
    ax.bar(ind + width/2, data.MedianPrice, width,
                label='Cost')
    
    ax.bar(ind - width/2, data['Accumulated Revenue'], width,
                label='Revenue')
    
    ax.bar(ind - width*1.5, data['Profit'], width,
                label='Profit')

    ax.set_ylabel('Dollars ($USD)', fontsize = 17.5)
                               
    ax.set_xlabel(f'Zipcodes for {location}', fontsize = 17.5)
                               
    ax.set_title(f'Forecasted Profit Analysis Over {years} Years for {bedrooms} bedrooms in {location}', fontsize = 20)
                               
    ax.set_xticks(ind)
                               
    ax.set_xticklabels(data.zipcode, rotation = (45))

    #  Scalable yticks rounded to the 10,000's place
    ytick_values = [*range(0, n := int(max(data.MedianPrice)), round(n//8, -5))]
    ytick_labels = ["${:,}".format(i) for i in ytick_values]
    
    plt.yticks(ytick_values, ytick_labels)

        
    ax.legend()

    plt.show()

interactive(children=(IntSlider(value=10, description='years', max=30, min=-10), Output()), _dom_classes=('wid…

<h3>Visual Narrative (3/3)</h3>
<br><br>
<text>
3. <b>Dynamic Data Table</b> 
<br><br>
No analysis is complete without a peak into the numbers. The dynamic data table allows for numerical analysis to be seen in real time.
<br><br>
&emsp;This tables shows three main interactive metrics:
<br><br>
&emsp;&emsp;1. Accumulated Revenue: defined above
<br><br>
&emsp;&emsp;2. Profit: defined above
<br><br>
&emsp;&emsp;3. break_even: the number of years remaining for a zipcode to become profitable
<br><br>
&emsp;break_even can be defined as below:
<br><br>
$$ Break Even = \frac {Median Price - Accumulated Revenue} {Revenue Per Year} $$
    

In [23]:
@widgets.interact(data = widgets.fixed(pull_data(**search_terms)),
                  Years = '0')
def breakeven(data, Years) -> pd.DataFrame:  
    """
    Creates Dynamic Break Even Analysis DataFrame
    
    Parameters
    ----------
    
    data: Returned from pull_data function
    Years: Dynamic metric for calculating projected profitability
    
    """
    
    try:
        
        if int(Years) < 0:
        
            return 'Please only use positive numbers'
        
    except ValueError:
            
        raise Exception('Please use positive integers')
    
    data['Accumulated Revenue'] = round(data.revenue_per_year * int(Years), 2)
    
    #  Calculate Profit
    data['Profit'] = ["${:,}".format(profit) if (profit := round(data['Accumulated Revenue'][i] - data['MedianPrice'][i], 2)) > 0 else
                     "Not Yet Profitable" for i in range(data.shape[0])]
    
    #  Years left until profitability
    data['break_even'] = ["Paid Off" if data['Accumulated Revenue'][i] >= data['MedianPrice'][i] else 
                           f"{round((data['MedianPrice'][i] - data['Accumulated Revenue'][i]) / data['revenue_per_year'][i], 2)} years left" 
                           for i in range(data.shape[0])]
    
    #  Sort by zipcodes paid off
    data['sorter'] = [0 if data['Accumulated Revenue'][i] >= data['MedianPrice'][i] else 
                    1 for i in range(data.shape[0])]
    
    data = data.sort_values(by='sorter').drop('sorter', axis = 1)
        
    return data

interactive(children=(Text(value='0', description='Years'), Output()), _dom_classes=('widget-interact',))

<h3>Final Analysis</h3>
<br><br>
<text>
    Zipcode <b> 10312 </b> overwhelmingly provides the greatest return on investment for the available zipcodes in New York City. The top 5 most profitable zipcodes in NYC after 30 years are as follows:
    
    1. 10312
    2. 10036
    3. 10022
    4. 10025
    5. 10011

    A more robust dataset with complete zipcode level data could change the results of this analysis. 

With the availablity of more data in the future, frameworks would need to be put in place to handle mass data uploads and scalability. I would also recommend a central database for more efficient data extraction. 
    
If additional Zillow data was made available for properties with a different number of bedrooms, the <b>pull_data</b> function would need to be updated to query the Zillow data by number of bedrooms.